In [6]:
import duckdb as dk
import pandas as pd

con = dk.connect('data/vermont.duckdb')

In [11]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [13]:
con.execute("""--sql
DROP TABLE IF EXISTS rolling_avg_freq;
CREATE TABLE rolling_avg_freq AS
WITH wrapped AS (
    SELECT
        *,
        day_of_year AS wrapped_day_of_year
    FROM detection_frequencies

    UNION ALL

    SELECT
        *,
        day_of_year + 366 AS wrapped_day_of_year
    FROM detection_frequencies
    WHERE day_of_year <= 6
),

rolling AS (
    SELECT
        locality,
        locality_id,
        day_of_year,
        common_name,
        total_detections,
        SUM(total_detections) OVER w AS k,
        total_checklists,
        SUM(total_checklists) OVER w AS n
    FROM wrapped
    WHERE 4 <= wrapped_day_of_year AND wrapped_day_of_year <= 369
    WINDOW w AS (
        PARTITION BY locality_id, common_name
        ORDER BY wrapped_day_of_year
        RANGE BETWEEN 3 PRECEDING AND 3 FOLLOWING
    )
    ORDER BY day_of_year
)

SELECT
    locality,
    locality_id,
    day_of_year,
    common_name,
    k::INT AS rolling_detections,
    n::INT AS rolling_checklists,
    k::DOUBLE / n AS rolling_freq,
    ((k::DOUBLE / n)
        + (1.64 * 1.64) / (2 * n)
        - 1.64 * SQRT(
            ((k::DOUBLE / n) * (1 - (k::DOUBLE / n)) / n)
            + ((1.64 * 1.64) / (4 * n * n))
        )
    )
    /
    (1 + (1.64 * 1.64) / n) AS wilson_lower_bound
    FROM rolling
;
""")

In [24]:
con.execute("SELECT * FROM rolling_avg_freq WHERE locality == 'Carse Wetlands (UVM Natural Area)' AND common_name =='Black-capped Chickadee'").fetchdf()

,locality,locality_id,day_of_year,common_name,rolling_detections,rolling_checklists,rolling_freq,wilson_lower_bound
0,Carse Wetlands (UVM Natural Area),5912437,14,Black-capped Chickadee,0,1,0.00,0.00
1,Carse Wetlands (UVM Natural Area),5912437,46,Black-capped Chickadee,1,1,1.00,0.27
2,Carse Wetlands (UVM Natural Area),5912437,53,Black-capped Chickadee,1,1,1.00,0.27
3,Carse Wetlands (UVM Natural Area),5912437,69,Black-capped Chickadee,0,1,0.00,0.00
4,Carse Wetlands (UVM Natural Area),5912437,74,Black-capped Chickadee,3,3,1.00,0.53
5,Carse Wetlands (UVM Natural Area),5912437,77,Black-capped Chickadee,5,6,0.83,0.50
6,Carse Wetlands (UVM Natural Area),5912437,78,Black-capped Chickadee,3,5,0.60,0.27
7,Carse Wetlands (UVM Natural Area),5912437,79,Black-capped Chickadee,4,6,0.67,0.35
8,Carse Wetlands (UVM Natural Area),5912437,81,Black-capped Chickadee,5,7,0.71,0.41
9,Carse Wetlands (UVM Natural Area),5912437,82,Black-capped Chickadee,4,6,0.67,0.35
